In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# IMPORTAMOS LOS DF
train_df = pd.read_csv('../data/train.csv')

# COLUMNAS CON CARACTERÍSTICAS DEL LIBRO (GÉNEROS Y PALABRAS DE LAS REVIEWS)
genre_columns = [col for col in train_df.columns if col.startswith('genre_')]
word_desc_columns = [col for col in train_df.columns if col.startswith('word_desc_')]

# Excluimos columnas que no son necesarias para la similitud, como 'book_title', 'reviewer', 'reviewer_rating'
excluded_columns = ['book_title', 'reviewer', 'reviewer_rating']
tfidf_columns = [col for col in train_df.columns if col not in excluded_columns]

# Solo seleccionamos las columnas numéricas o booleanas (géneros + palabras de reviews)
similarity_data = train_df[tfidf_columns].copy()

# Convertimos las columnas booleanas a enteros de manera explícita
boolean_columns = similarity_data.select_dtypes(include=['bool']).columns.tolist()
similarity_data[boolean_columns] = similarity_data[boolean_columns].astype(int)

# SIMILITUD DE COSENO ENTRE LIBROS (incluir géneros y palabras de reviews)
cosine_sim = cosine_similarity(similarity_data)

# FUNCIÓN PARA RECOMENDAR LIBROS BASADO EN TODOS LOS LIBROS CALIFICADOS CON 4 o 5 POR UN REVISOR
def recommend_books_for_reviewer(reviewer, train_df, cosine_sim_matrix, num_recommendations=5):
    # Filtramos SOLO los libros que el revisor específico calificó con 4 o más
    user_books = train_df[(train_df['reviewer'] == reviewer) & (train_df['reviewer_rating'] >= 4)]
    
    # Eliminar duplicados basados en el 'book_title', no en los índices
    user_books = user_books.drop_duplicates(subset='book_title')
    
    # Debug: Mostramos todos los libros que el usuario ha calificado con 4 o más
    print(f"Libros calificados por {reviewer} con 4 o más estrellas (sin duplicados por título):\n", user_books[['book_title', 'reviewer_rating']])
    
    if user_books.empty:
        return "No se encontraron libros calificados con 4 o más para este revisor."
    
    # Obtenemos los índices de los libros que ha calificado el usuario
    book_indices = user_books.index.tolist()
    
    # Calculamos el promedio de las similitudes entre todos los libros que el revisor calificó con 4 o más
    sim_scores = cosine_sim_matrix[book_indices].mean(axis=0)  # Promedio de las similitudes
    
    # Ordenamos los libros más similares
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    unique_recommendations = set()
    recommendations_with_scores = []
    
    for i in range(len(sim_scores)):
        book_idx = sim_scores[i][0]
        recommended_title = train_df['book_title'].iloc[book_idx]
        recommended_rating = train_df['rating'].iloc[book_idx]  # Calificación promedio del libro
        
        # Evitar recomendar libros ya leídos por el usuario y con rating promedio menor a 4.5
        if (recommended_title not in user_books['book_title'].values 
            and recommended_title not in unique_recommendations
            and recommended_rating >= 4.5):  # Filtramos por calificación promedio >= 4.5
            unique_recommendations.add(recommended_title)
            recommendations_with_scores.append((recommended_title, sim_scores[i][1]))
        
        if len(recommendations_with_scores) >= num_recommendations:
            break

    return recommendations_with_scores

# FUNCIÓN PARA MOSTRAR LOS RESULTADOS CON PORCENTAJES DE SIMILITUD
def show_recommendations_with_similarity(reviewer, train_df, cosine_sim_matrix, recommendations):
    # Filtramos los libros que el revisor calificó con 4 o más
    user_books = train_df[(train_df['reviewer'] == reviewer) & (train_df['reviewer_rating'] >= 4)]
    
    if user_books.empty:
        print("No se encontraron libros calificados con 4 o más para este revisor.")
        return
    
    # Eliminamos duplicados de los libros que le gustaron al usuario (basado en 'book_title')
    user_books = user_books.drop_duplicates(subset='book_title')
    liked_books_titles = user_books['book_title'].tolist()  # Todos los libros que le gustaron al usuario
    
    print(f"Recomendaciones basadas en los libros que le gustaron al usuario: {', '.join(liked_books_titles)}")
    
    for recommended_title, similarity_score in recommendations:
        similarity_percentage = similarity_score * 100  # Convertimos la similitud en porcentaje
        print(f"- {recommended_title} (Similitud: {similarity_percentage:.2f}%)")

# EJEMPLO: RECOMENDACIONES PARA UN REVISOR ESPECÍFICO
reviewer_to_recommend = 'Ashley'
recommendations = recommend_books_for_reviewer(reviewer_to_recommend, train_df, cosine_sim, num_recommendations=5)

# MOSTRAR RECOMENDACIONES CON PORCENTAJE DE SIMILITUD
if recommendations:
    show_recommendations_with_similarity(reviewer_to_recommend, train_df, cosine_sim, recommendations)


Libros calificados por Ashley con 4 o más estrellas (sin duplicados por título):
                                              book_title  reviewer_rating
643   Stocking Stuffer Mad Libs: World's Greatest Wo...                5
1972              Haunting Adeline (Cat and Mouse Duet)                4
Recomendaciones basadas en los libros que le gustaron al usuario: Stocking Stuffer Mad Libs: World's Greatest Word Game, Haunting Adeline (Cat and Mouse Duet)
- Dirty Thirty (30) (Stephanie Plum) (Similitud: 94.62%)
- Goodnight Moon (Similitud: 93.99%)
- How to Catch a Reindeer (Similitud: 93.66%)
- Hello, Baby Animals: A Durable High-Contrast Black-and-White Board Book for Newborns and Babies (High-Contrast Books) (Similitud: 93.65%)
- Where's Bluey?: A Search-and-Find Book (Similitud: 93.30%)
